In [4]:
import getpass
import os


def _set_env(key: str):
    if key not in os.environ:
        os.environ[key] = getpass.getpass(f"{key}:")


_set_env("OPENAI_API_KEY")

In [5]:
from typing import Annotated, Sequence
from typing_extensions import TypedDict

from langchain_core.messages import BaseMessage

from langgraph.graph.message import add_messages


class AgentState(TypedDict):
    # The add_messages function defines how an update should be processed
    # Default is to replace. add_messages says "append"
    messages: Annotated[Sequence[BaseMessage], add_messages]

I am working on keeping separate nodes for:
1. Class extraction,
2. Rubric extraction,
3. Initial evaluation,
4. Evaluation review,
5. Marks extraction, and
6. Marks calculation.

In [12]:
# tool for marks calculation

# @tool
def sum_marks(marks):
    """
    Description:
        Return sum of sequence of marks.
    
    Args:
        marks (str): Comma separated string of marks
        
    Returns:
        sum_of_marks (int): Sum of marks"""
    
    sum_of_marks = sum(int(mark) for mark in marks)
    return sum_of_marks

In [20]:
from typing import Annotated, Literal, Sequence
from typing_extensions import TypedDict

from langchain import hub
from langchain_core.messages import BaseMessage, HumanMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

from pydantic import BaseModel, Field


from langgraph.prebuilt import tools_condition

### Nodes

def parse_classes(state):
    """
    Parses given student Java code submission and extracts individual classes. Also parses given model Java code and extracts individual classes.

    Args:
        state (messages): The current state

    Returns:
        dict: The updated state with the agent response appended to messages. Agent response should look like: (List[str], List[str]): Tuple where first element is list of strings from student submission where each string is a separate class, and second element is list of strings from model code where each string is a separate class.
    """

    print("---PARSE CLASSES---")

    # LLM
    messages = state["messages"]
    model_1 = ChatOpenAI(temperature=0, model="gpt-4-0125-preview", streaming=True)
    response = model_1.invoke(messages)
    # We return a list, because this will get added to the existing list
    return {"messages": [response]}

def parse_rubric(state):
    """
    Parses given rubric and separate out rubric elements for different classes (if there is more than 1 class).

    Args:
        state (messages): The current state

    Returns:
        dict: The updated state with the agent response appended to messages. Agent response should look like: List[str]: List of strings where each string is the rubric for a certain class.
    """

    print("---PARSE RUBRIC---")

    # LLM
    messages = state["messages"]
    model_2 = ChatOpenAI(temperature=0, model="gpt-4-0125-preview", streaming=True)
    response = model_2.invoke(messages)
    # We return a list, because this will get added to the existing list
    return {"messages": [response]}

def evaluate(state):
    """
    Given a student's submission, a model solution and a rubric, evaluates the submission by assigning scores for each rubric, while giving detailed comments about the correctness, errors and suggestions for improvement in the student code.

    Args:
        state (messages): The current state

    Returns:
        dict: The updated state with the agent response appended to messages. Agent response should include scores for each rubric, followed by required comments.
    """
    print("---EVALUATE---")
    messages = state["messages"]
    model_3 = ChatOpenAI(temperature=0, streaming=True, model="gpt-4o")
    response = model_3.invoke(messages)
    # We return a list, because this will get added to the existing list
    return {"messages": [response]}

def review(state):
    """
    Given an evaluation of a student's submission, a model solution and a rubric, reviews the submission and corrects it where necessary.

    Args:
        state (messages): The current state

    Returns:
        dict: The updated state with the agent response appended to messages.
    """
    print("---REVIEW---")
    messages = state["messages"]
    model_4 = ChatOpenAI(temperature=0, streaming=True, model="gpt-4o")
    response = model_4.invoke(messages)
    # We return a list, because this will get added to the existing list
    return {"messages": [response]}

def extract_marks(state):
    """
    Given an evaluation of a student's submission, a model solution and a rubric, reviews the submission and corrects it where necessary.

    Args:
        state (messages): The current state

    Returns:
        dict: The updated state with the agent response appended to messages. Agent response should look like: List[int]: List of integers where each integer is the score for a certain rubric.
    """
    print("---EXTRACT MARKS---")
    messages = state["messages"]
    model_5 = ChatOpenAI(temperature=0, streaming=True, model="gpt-4o")
    response = model_5.invoke(messages)

    with open("final_evaluations.txt", "w") as file:
        # Writing data to a file
        file.write(response.content + "\n\n")


    # We return a list, because this will get added to the existing list
    return {"messages": [response]}

from langgraph.prebuilt import ToolNode

def calculate_marks(state):
    """
    Given a list of integers of marks, returns sum of marks.

    Args:
        state (messages): The current state

    Returns:
        dict: The updated state with the agent response appended to messages. Agent response should look like: sum_of_marks (int): The sum of the given marks.
    """
    print("---SUM MARKS---")
    messages = state["messages"]
    tools = [sum_marks]
    model_6 = ChatOpenAI(temperature=0, streaming=True, model="gpt-4o").bind_tools(tools)
    response = model_6.invoke(messages)

    with open("final_evaluations.txt", "a") as file:
        # Writing data to a file
        file.write(f"Final score: {response.content} \n\n")

    # We return a list, because this will get added to the existing list
    return {"messages": [response]}

In [21]:
from langgraph.graph import END, StateGraph, START
from langgraph.prebuilt import ToolNode

# Define a new graph
workflow = StateGraph(AgentState)

# Define the nodes we will cycle between
# sum_of_marks = ToolNode([sum_marks])
workflow.add_node("parse_classes", parse_classes)  # retrieval
workflow.add_node("parse_rubric", parse_rubric)  # Re-writing the question
workflow.add_node("evaluate", evaluate)  # retrieval
workflow.add_node("review", review)  # retrieval
workflow.add_node("extract_marks", extract_marks)  # retrieval
workflow.add_node("calculate_marks", calculate_marks)  # retrieval

workflow.add_edge(START, "parse_classes")
workflow.add_edge("parse_classes", "parse_rubric")
workflow.add_edge("parse_rubric", "evaluate")
workflow.add_edge("evaluate", "review")
workflow.add_edge("review", "extract_marks")
workflow.add_edge("extract_marks", "calculate_marks")
workflow.add_edge("calculate_marks", END)
    
# Compile
graph = workflow.compile()

In [22]:
def stream_graph_updates(user_input: str):
    for event in graph.stream({"messages": [("user", user_input)]}):
        for value in event.values():
            print("Assistant:", value["messages"][-1].content)


with open("/Users/deepan_roy/Desktop/m24-midsem-install-main/venv_mid_sem_exam/simple_scenario/model_solution.md", "r") as file:
    model_solution = file.read()

with open("/Users/deepan_roy/Desktop/m24-midsem-install-main/venv_mid_sem_exam/simple_scenario/question.md", "r") as file:
    question = file.read()

with open("/Users/deepan_roy/Desktop/m24-midsem-install-main/venv_mid_sem_exam/simple_scenario/rubric.md", "r") as file:
    rubric = file.read()

with open("/Users/deepan_roy/Desktop/m24-midsem-install-main/venv_mid_sem_exam/simple_scenario/student_solution.md", "r") as file:
    student_solution = file.read()


user_input = f"Model Solution: {model_solution} \n\n\n Question: {question} \n\n\n Rubric: {rubric} \n\n\n Student Solution: {student_solution}"
stream_graph_updates(user_input)

---PARSE CLASSES---
